In [1]:
import open3d as o3d
import numpy as np
import os
import glob
from tqdm import tqdm
from time import time
import subprocess
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Convert mesh to a point cloud and estimate dimensions.
armadillo_data = o3d.data.ArmadilloMesh()
pcd = o3d.io.read_triangle_mesh(
    armadillo_data.path).sample_points_poisson_disk(5000)
diameter = np.linalg.norm(
    np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
print("Diameter:", diameter)
print("Displaying input point cloud ...")
# o3d.visualization.draw([pcd], point_size=5)

Diameter: 227.86129651492362
Displaying input point cloud ...
[Open3D INFO] Window window_1 created.


KeyboardInterrupt: 

In [2]:
def sub_sample_cloud(input_file: str, output_dir: str):
    
    os.makedirs(output_dir, exist_ok=True)
    
    input_file_name = os.path.basename(input_file).split('.')[0]
    
    # Define the output filenames with modulo values
    modulo_values = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]
    
    # Process the file
    with open(input_file, "r") as infile:
        lines = infile.readlines()
    
    # Write lines based on modulo conditions
    for mod_value in tqdm(modulo_values):
        output_file = os.path.join(output_dir, f"{input_file_name}_mod_40{mod_value}.txt")
        with open(output_file, "w") as outfile:
            # Filter lines where the line number modulo 40 equals mod_value
            outfile.writelines(line for idx, line in enumerate(lines, start=1) if idx % 40 == mod_value)

def create_shadow_data(area_name: str, sub_sampled_dir:str, output_dir: str, interval: int=10, mod_val: int=20, radius: int=10000):
    pcds = []
    def check_dimensions(pcd):        
        size = pcd.get_max_bound() - pcd.get_min_bound()
        if size[0]//interval < 2.0 or size[1]//interval < 2.0 or size[2]//interval < 2.0:
            raise ValueError("Building Dimension: {}. But dimension in any axis must be greater than twice the interval value: {}".format(size, interval))
        else:
            return True
    size_ok = False
    for sub_sampled_file in os.listdir(sub_sampled_dir):
        if not size_ok:
            size_ok = check_dimensions(o3d.io.read_point_cloud(os.path.join(sub_sampled_dir, sub_sampled_file), format='xyz', print_progress=True))
        if size_ok:
            pcds.append(o3d.io.read_point_cloud(os.path.join(sub_sampled_dir, sub_sampled_file), format='xyz', print_progress=True))
            print("Loaded", sub_sampled_file)
    if len(pcds) > 0:
        max = pcds[0].get_max_bound()
        min = pcds[0].get_min_bound()
        building_size = max-min
        print("Building Size [x, y, z]: {}".format(building_size))
        # z_pts = np.array([40, 90, 140]) # manual
        z_pts = interval * np.arange(building_size[2] // interval)[1:]
        y_pts = interval * np.arange(building_size[1] // interval)[1:]
        x_pts = interval * np.arange(building_size[0] // interval)[1:]
        print("x intervals at ", x_pts)
        print("y intervals at ", y_pts)
        print("z intervals at ", z_pts)
        
        i = 0 
        for cam_x in x_pts:
            for cam_y in y_pts:
                for cam_z in z_pts:
                    print("i=",i)
                    camera = [min[0]+cam_x, min[1]+cam_y, min[2]+cam_z]
                    print("camera at ",camera)
                    index = np.random.randint(mod_val)
                    print("index: ",index)
                    _, pt_map = pcds[index].hidden_point_removal(camera, radius)
                    pcd_new = pcds[index].select_by_index(pt_map)
                    os.makedirs(os.path.join(output_dir, area_name), exist_ok=True)
                    file_name = os.path.join(output_dir,area_name,"{}_{}.pts".format(area_name, i))
                    print("Writing", file_name)
                    o3d.io.write_point_cloud(file_name, pcd_new)
                    i = i+1

def get_file_name(file_path: str) -> str:
    return os.path.basename(file_path).split('.')[0]

In [3]:
def get_max_min_dim(file_path: str, dim_size = 0.01):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, skiprows=1, encoding="gbk", engine='python', sep=' ', delimiter=None, index_col=False, header=None, skipinitialspace=True)
        
        xMin = df[0].min()
        xMax = df[0].max()
        xDim = xMax - xMin
        xMinLimit = xMin - (xDim * dim_size)
        xMaxLimit = xMax + (xDim * dim_size)
        
        yMin = df[1].min()
        yMax = df[1].max()
        yDim = yMax - yMin
        yMinLimit = yMin - (yDim * dim_size)
        yMaxLimit = yMax + (yDim * dim_size)
       
        zMin = df[2].min()
        zMax = df[2].max()
        zDim = zMax - zMin
        zMinLimit = zMin - (zDim * dim_size)
        zMaxLimit = zMax + (zDim * dim_size)
    
        print(xMinLimit, xMaxLimit, yMinLimit, yMaxLimit, zMinLimit, zMaxLimit)
        limits = []
        limits.append(xMinLimit)
        limits.append(xMaxLimit)
        limits.append(yMinLimit)
        limits.append(yMaxLimit)
        limits.append(zMinLimit)
        limits.append(zMaxLimit)
        suffix = file_path+'.txt'
        with open(suffix, 'w') as file:
            for limit in limits:
                file.write(str(limit))
                file.write(' ')

def extract_pcd_from_mesh(area_name: str, fbx_dir: str, fbx_classes: list, temp_folder: str, output_dir: str):
    # Define paths
    cloud_compare_path = r"c:\program files\cloudcompare\cloudcompare.exe"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    os.makedirs(temp_folder, exist_ok=True)
    for fbx_class in fbx_classes:
        print("Extracting from ", fbx_class)
        # Loop through all .fbx files in the input folder
        fbx_files = glob.glob(os.path.join(fbx_dir, fbx_class, "*.fbx"))
        if len(fbx_files) > 0:
            print("Found {} fbx files for class {}".format(len(fbx_files), fbx_class))
            for fbx_file in fbx_files:
                file_name = get_file_name(fbx_file)
                # Generate intermediate filenames
                merged_bin = os.path.join(temp_folder, file_name+"_merged.bin")
                csv_file = os.path.join(temp_folder, file_name+".csv")
                
                # Step 1: Sample mesh and generate merged binary
                subprocess.run([
                    cloud_compare_path, "-silent", "-auto_save", "off", "-o", fbx_file,
                    "-sample_mesh", "density", "500", "-merge_clouds", "-save_clouds",
                    "file", merged_bin, "-clear", "-o", merged_bin,
                    "-c_export_fmt", "asc", "-add_header", "-ext", "csv", "-save_clouds", "file", csv_file
                ])
                
                get_max_min_dim(csv_file)
                
                # Read the modified CSV file and process tokens
                txt_file = f"{csv_file}.txt"
                if os.path.exists(txt_file):
                    with open(txt_file, "r") as file:
                        for line in file:
                            tokens = line.split()  # Split line into tokens
                            if len(tokens) >= 6:
                                a, b, c, d, e, f = tokens[:6]
                                
                                # Step 3: Crop and save cropped binary
                                cropped_bin =  os.path.join(temp_folder, file_name+"_cropped_pt01.bin")
                                consolidated_bin = os.path.join(temp_folder, file_name+"_consolidated.bin")
                                subprocess.run([
                                    cloud_compare_path, "-silent", "-auto_save", "off",
                                    "-o", consolidated_bin, "-crop",
                                    f"{a}:{c}:{e}:{b}:{d}:{f}", "-save_clouds", "file", cropped_bin
                                ])
                                
                                # Step 4: Export cropped binary to .pts format
                                output_path = os.path.join(output_dir, area_name+".pts")
                                subprocess.run([
                                    cloud_compare_path, "-silent", "-auto_save", "off",
                                    "-o", cropped_bin, "-c_export_fmt", "asc",
                                    "-add_header", "-ext", "pts", "-save_clouds", "file", output_path
                                ])

In [7]:
MASTER_ROOT = "Master"
sub_sampled_dir = "sub_sampled"
shadow_output_dir = "shadow_xyz"
final_output_dir = "Shadow"
fbx_dir = "Fbx_Wall"
fbx_classes = ['floor', 'beam', 'roof', 'wall', 'column', 'door', 'window', 'busbar', 'cable', 'duct', 'pipe']

AREAS = os.listdir(MASTER_ROOT)
master_files = []
for area in AREAS:
    files = os.listdir(os.path.join(MASTER_ROOT, area))
    for file in files:
        if file == area+'.txt':
            master_files.append(os.path.join(MASTER_ROOT,area,file))
print(f"Found {len(master_files)} files from {len(AREAS)} areas in {MASTER_ROOT}")

Found 1 files from 1 areas in Master


In [9]:
failures = []
for master_file in master_files:
    start_time = time()
    area_name = get_file_name(master_file)
    # print("Subsampling", master_file)
    # sub_sample_cloud(master_file, str(os.path.join(sub_sampled_dir, area_name)))
    # print("Creating shadow pcd of", master_file)
    # try:
    #     create_shadow_data(area_name, str(os.path.join(sub_sampled_dir, area_name)), shadow_output_dir)
    #     end_time = time()
    #     elapsed_time = (end_time - start_time)/60   # in minutes
    #     print("Shadow data of {} created in {} minutes".format(master_file, elapsed_time))
    # except:
    #     failures.append(master_file)
    #     continue
    print("Creating final Shadow data of {} with RGB fields".format(master_file))
    extract_pcd_from_mesh(area_name, fbx_dir, fbx_classes, "temp_data", final_output_dir)
if len(failures) > 0:
    print("Failed to create shadow data for")
    for failure in failures:
        print(failure)

Creating final Shadow data of Master\Area_2_SB2_part_1\Area_2_SB2_part_1.txt with RGB fields
Extracting from  floor
Extracting from  beam
Extracting from  roof
Extracting from  wall
Found 483 fbx files for class wall
373.1719049072266 373.8412176513674 644.8713232421873 668.1627343749998 29.870407543182107 45.09678415298489
530.7047613525391 544.4084588623049 726.2938531494145 726.9631658935544 29.88025007247899 44.10268869400001
535.9568951416018 544.3566424560552 667.9278253173826 668.5971380615234 29.88025007247899 44.10268869400001
535.3339947509766 543.7001239013674 645.0931091308591 645.7623596191409 29.870407543182107 45.09678415298489
543.6115411376956 544.2808538818364 644.8713232421873 668.1627343749998 29.870407543182107 45.09678415298489
375.37006591796916 384.4055078124999 749.0957336425781 749.7649841308598 44.82119361877468 57.70505149841332
383.65420501709036 384.32348663330066 730.7446942138674 749.2840533447265 44.81299152374294 58.53346309661906
373.0441162109375 386